In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import os
from glob import glob

# 파일 경로 설정
tidal_current_path = "tidal_current_2023.csv" # 해류 데이터
# marideb_timestep_path = "marideb_timestep.csv"
grid_path = "grid_latlon.csv" # 기존 격자 데이터
debris_path = "marideb_location.csv"
sden_path = "ship_density_aggregated_2023.csv"


# 데이터 로드
tidal_current_df = pd.read_csv(tidal_current_path)   # 해류 데이터
sden_df = pd.read_csv(sden_path)  # 선박 밀집도 데이터
deb_df = pd.read_csv(debris_path) #해안쓰레기 데이터
grid_df = pd.read_csv(grid_path)   # 기존 격자 데이터



In [2]:
# 📌 컬럼명 변경 (한글 → 영어)
deb_df.rename(columns={
    "정점명": "Checkpoint",
    "개수": "Waste_Count",
    "무게_kg": "Waste_Weight",
    "위도": "Latitude",
    "경도": "Longitude",
    "군집": "Cluster",
    "단위무게": "Unit_Weight"
}, inplace=True)

# 📌 변경된 컬럼 확인
print(deb_df.columns)


tidal_current_df.rename(columns={
    "pre_lat": "Latitude",
    "pre_lon": "Longitude"
}, inplace=True)


Index(['Checkpoint', 'Waste_Count', 'Waste_Weight', 'Latitude', 'Longitude',
       'Cluster', 'Unit_Weight'],
      dtype='object')


In [3]:
deb_df.info()
tidal_current_df.info()
sden_df.info()
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Checkpoint    60 non-null     object 
 1   Waste_Count   60 non-null     int64  
 2   Waste_Weight  60 non-null     float64
 3   Latitude      60 non-null     float64
 4   Longitude     60 non-null     float64
 5   Cluster       60 non-null     int64  
 6   Unit_Weight   60 non-null     float64
dtypes: float64(4), int64(2), object(1)
memory usage: 3.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4520160 entries, 0 to 4520159
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   current_dir    int64  
 1   Longitude      float64
 2   current_speed  float64
 3   Latitude       float64
 4   datetime       object 
dtypes: float64(3), int64(1), object(1)
memory usage: 172.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30867963 entries, 0 to 3086

In [5]:
print(deb_df.head())
print(tidal_current_df.head())
print(sden_df.head())
print(grid_df.head())

  Checkpoint  Waste_Count  Waste_Weight   Latitude   Longitude  Cluster  \
0      강화여차리          511          25.1  37.620648  126.389360        2   
1      안산말부흥           50           0.5  37.221114  126.572177        2   
2      태안백리포         1359          49.8  36.814035  126.159965        2   
3      보령석대도          138          50.1  36.247135  126.546496        2   
4       부안변산           97          39.0  35.689892  126.549436        2   

   Unit_Weight  
0     0.049119  
1     0.010000  
2     0.036645  
3     0.363043  
4     0.402062  
   current_dir  Longitude  current_speed  Latitude             datetime
0          147  125.03305           43.0  33.04701  2023-01-01 00:00:00
1          146  125.02657           44.0  33.19098  2023-01-01 00:00:00
2          147  125.02003           45.0  33.33496  2023-01-01 00:00:00
3          147  125.01345           43.0  33.47892  2023-01-01 00:00:00
4          147  125.00682           42.0  33.62288  2023-01-01 00:00:00
       Grid_ID 

In [6]:
import pandas as pd
import numpy as np
import gc  # 가비지 컬렉션 사용

# 1. 먼저 모든 float64를 float32로 변환하여 메모리 사용량 줄이기
def convert_to_float32(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    return df

# float32로 변환 적용
print("float32로 변환 중...")
tidal_current_df = convert_to_float32(tidal_current_df)
sden_df = convert_to_float32(sden_df)
deb_df = convert_to_float32(deb_df)
grid_df = convert_to_float32(grid_df)
print("변환 완료")

# 2. 데이터 조인 전략
# - grid_df는 기준 격자 정보
# - sden_df는 선박 밀집도 (Grid_ID와 datetime으로 연결)
# - tidal_current_df는 해류 데이터 (위치와 datetime으로 연결)
# - deb_df는 해안쓰레기 데이터 (위치로 연결)

# 2-1. grid_df를 기준으로 시작
print("grid_df와 sden_df 조인 중...")

# grid_df에 있는 모든 Grid_ID 보존하기
grid_ids = set(grid_df['Grid_ID'])

# sden_df를 Grid_ID 기준으로 필터링
sden_filtered = sden_df[sden_df['Grid_ID'].isin(grid_ids)]

# 메모리 확보를 위해 원본 sden_df 삭제
del sden_df
gc.collect()

# 데이터가 너무 많으면 일부 시간대만 선택할 수도 있음
# 예: 특정 날짜/시간만 선택
unique_datetimes = sden_filtered['datetime'].unique()
print(f"고유 시간대 수: {len(unique_datetimes)}")

# 시간대별로 처리하는 방식으로 병합
merged_results = []

# 시간 단위로 처리
num_chunks = min(20, len(unique_datetimes))  # 너무 많은 청크는 피함
time_chunks = np.array_split(unique_datetimes, num_chunks)

for i, time_chunk in enumerate(time_chunks):
    print(f"시간 청크 {i+1}/{len(time_chunks)} 처리 중...")
    
    # 해당 시간대의 sden 데이터만 선택
    chunk_sden = sden_filtered[sden_filtered['datetime'].isin(time_chunk)]
    
    # 해당 시간대의 tidal_current 데이터만 선택
    chunk_tidal = tidal_current_df[tidal_current_df['datetime'].isin(time_chunk)]
    
    # 2-2. grid_df와 chunk_sden 병합 (Grid_ID 기준)
    merged_df = pd.merge(
        grid_df,
        chunk_sden,
        on='Grid_ID',
        how='left'  # grid_df의 모든 Grid_ID 보존
    )
    
    # 2-3. 위치 기반 조인을 위한 준비
    # tidal_current와 합치기 위해 가장 가까운 좌표 찾기
    # 복잡한 공간 조인 대신 간단화된 방법 사용
    
    # 먼저 각 시간대별로 처리
    for dt in time_chunk:
        tidal_at_time = chunk_tidal[chunk_tidal['datetime'] == dt]
        merged_at_time = merged_df[merged_df['datetime'] == dt]
        
        if len(tidal_at_time) == 0 or len(merged_at_time) == 0:
            continue
            
        # 각 격자점에 대해 가장 가까운 해류 데이터 찾기
        # 이 부분은 계산 비용이 높음 - 최적화 필요
        for idx, grid_row in merged_at_time.iterrows():
            # 거리 계산 (단순화된 방법)
            distances = (
                (tidal_at_time['Latitude'] - grid_row['Latitude'])**2 + 
                (tidal_at_time['Longitude'] - grid_row['Longitude'])**2
            )
            nearest_idx = distances.idxmin()
            
            # 가장 가까운 tidal 데이터의 값 추가
            nearest_tidal = tidal_at_time.loc[nearest_idx]
            merged_df.loc[idx, 'current_dir'] = nearest_tidal['current_dir']
            merged_df.loc[idx, 'current_speed'] = nearest_tidal['current_speed']
    
    # 중간 결과 저장
    merged_results.append(merged_df)
    
    # 메모리 정리
    del chunk_sden, chunk_tidal, merged_df
    gc.collect()

# 모든 중간 결과 병합
print("모든 결과 병합 중...")
final_merged = pd.concat(merged_results, ignore_index=True)
del merged_results
gc.collect()

# 2-4. 해안쓰레기 데이터 (deb_df) 병합
# 가장 가까운 격자점 찾기
print("해안쓰레기 데이터 병합 중...")
for idx, deb_row in deb_df.iterrows():
    distances = (
        (grid_df['Latitude'] - deb_row['Latitude'])**2 + 
        (grid_df['Longitude'] - deb_row['Longitude'])**2
    )
    nearest_idx = distances.idxmin()
    nearest_grid = grid_df.loc[nearest_idx]
    
    # 해당 Grid_ID에 해안쓰레기 정보 추가
    mask = final_merged['Grid_ID'] == nearest_grid['Grid_ID']
    if mask.any():
        final_merged.loc[mask, 'Waste_Count'] = deb_row['Waste_Count']
        final_merged.loc[mask, 'Waste_Weight'] = deb_row['Waste_Weight']
        final_merged.loc[mask, 'Cluster'] = deb_row['Cluster']
        final_merged.loc[mask, 'Unit_Weight'] = deb_row['Unit_Weight']

# 최종 결과 확인
print("최종 병합 완료")
print(f"최종 데이터프레임 크기: {final_merged.shape}")
print(f"grid_df의 모든 Grid_ID 포함 여부: {set(grid_df['Grid_ID']).issubset(set(final_merged['Grid_ID']))}")

float32로 변환 중...
변환 완료
grid_df와 sden_df 조인 중...
고유 시간대 수: 6619
시간 청크 1/20 처리 중...
시간 청크 2/20 처리 중...
시간 청크 3/20 처리 중...
시간 청크 4/20 처리 중...
시간 청크 5/20 처리 중...
시간 청크 6/20 처리 중...
시간 청크 7/20 처리 중...
시간 청크 8/20 처리 중...
시간 청크 9/20 처리 중...
시간 청크 10/20 처리 중...
시간 청크 11/20 처리 중...
시간 청크 12/20 처리 중...
시간 청크 13/20 처리 중...
시간 청크 14/20 처리 중...
시간 청크 15/20 처리 중...
시간 청크 16/20 처리 중...
시간 청크 17/20 처리 중...
시간 청크 18/20 처리 중...
시간 청크 19/20 처리 중...
시간 청크 20/20 처리 중...
모든 결과 병합 중...
해안쓰레기 데이터 병합 중...
최종 병합 완료
최종 데이터프레임 크기: (12449739, 19)
grid_df의 모든 Grid_ID 포함 여부: True


In [7]:
# 대용량 데이터를 청크 단위로 저장하는 방법
chunk_size = 500000  # 한 번에 저장할 행 수
output_path = "merged_data.csv"

# 첫 번째 청크는 헤더 포함
first_chunk = True
for i in range(0, len(final_merged), chunk_size):
    chunk = final_merged.iloc[i:i+chunk_size]
    
    if first_chunk:
        chunk.to_csv(output_path, mode='w', index=False, float_format='%.4f')
        first_chunk = False
    else:
        chunk.to_csv(output_path, mode='a', header=False, index=False, float_format='%.4f')
    
    print(f"청크 {i//chunk_size + 1}/{(len(final_merged) + chunk_size - 1)//chunk_size} 저장 완료")

print(f"데이터 저장 완료: {output_path}")

청크 1/25 저장 완료
청크 2/25 저장 완료
청크 3/25 저장 완료
청크 4/25 저장 완료
청크 5/25 저장 완료
청크 6/25 저장 완료
청크 7/25 저장 완료
청크 8/25 저장 완료
청크 9/25 저장 완료
청크 10/25 저장 완료
청크 11/25 저장 완료
청크 12/25 저장 완료
청크 13/25 저장 완료
청크 14/25 저장 완료
청크 15/25 저장 완료
청크 16/25 저장 완료
청크 17/25 저장 완료
청크 18/25 저장 완료
청크 19/25 저장 완료
청크 20/25 저장 완료
청크 21/25 저장 완료
청크 22/25 저장 완료
청크 23/25 저장 완료
청크 24/25 저장 완료
청크 25/25 저장 완료
데이터 저장 완료: merged_data.csv


In [8]:
final_merged.head()

,Grid_ID,Latitude,Longitude,datetime,avg_ship_density,avg_ship_count,avg_fishing,avg_passenger,avg_cargo,avg_tanker,avg_tugboat,avg_leisure,avg_other,current_dir,current_speed,Waste_Count,Waste_Weight,Cluster,Unit_Weight
0,GR3_G3B32_D,35.474998,129.425003,2023-01-01 01:00:00,45.560001,54.0,23.0,0.0,1.0,27.0,1.0,0.0,2.0,211.0,28.0,2280.0,27.700001,0.0,0.012149
1,GR3_G3B32_D,35.474998,129.425003,2023-01-01 02:00:00,45.279999,51.0,21.0,0.0,1.0,26.0,1.0,0.0,2.0,213.0,25.0,2280.0,27.700001,0.0,0.012149
2,GR3_G3B32_D,35.474998,129.425003,2023-01-01 03:00:00,45.830002,49.0,15.0,0.0,1.0,30.0,1.0,0.0,2.0,216.0,16.0,2280.0,27.700001,0.0,0.012149
3,GR3_G3B32_D,35.474998,129.425003,2023-01-01 04:00:00,44.439999,41.0,10.0,0.0,1.0,26.0,1.0,0.0,3.0,235.0,4.0,2280.0,27.700001,0.0,0.012149
4,GR3_G3B32_D,35.474998,129.425003,2023-01-01 06:00:00,42.020000,73.0,50.0,0.0,1.0,19.0,1.0,0.0,2.0,27.0,18.0,2280.0,27.700001,0.0,0.012149
